# Descarga de PDBs de la proteína ERK2
Esta libreta de Jupiter permite descargar las estructuras cristalográficas disponibles de una proteína dada una secuencia de UniProt, un valor de identidad y un valor de covertura.

**Como resultado**, al final de la ejecución todos los archivos *pdb* disponibles de la proteína son descargados, generándose tres archivos por cada **_pdb id_**:
1. Archivo en crudo del pdb id (el archivo tal y como se puede descargar del PDB).
2. Archivo pdb con únicamnete la cadena protéica correspondiente a la proteína de interés.
3. Archivo pdb con todas las moléculas con la etiqueta **HETATM** que acompañana a la proteína pudiendo ser moléculas pequeñas, iones o residuos portéicos modificados (las moléculas d eagua son removidas).

In [4]:
from prody import *
from pylab import *
import pickle

## Secuencia de la proteína
Se define el nombre de la proteína y su identificador en UNIPROT:

In [5]:
# Nombre de la proteina de interes
prot_name = 'erk2'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P28482"

Se obtiene la secuencia de la proteína y se guarda en un archivo en la carpeta *data*:

In [33]:
# Secuencia de la erk2 de UniProt
import requests
from Bio import SeqIO

# Descargamos el fasta de UniProt
url_fasta = requests.get("https://www.uniprot.org/uniprot/" + uniprot_id + ".fasta")
file_name_fasta = './data/' + uniprot_id + '.fasta'
open(file_name_fasta, 'wb').write(url_fasta.content)
# Leemos la secuenciade aminoácidos
fasta_prot = SeqIO.read(open(file_name_fasta),'fasta')
seq_prot = str(fasta_prot.seq)
print(seq_prot)
print(F'La longitud de la secuancia es: {len(seq_prot)} residuos.')

MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNVNKVRVAIKKISPFEHQTYCQRTLREIKILLRFRHENIIGINDIIRAPTIEQMKDVYIVQDLMETDLYKLLKTQHLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLLNTTCDLKICDFGLARVADPDHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINLKARNYLLSLPHKNKVPWNRLFPNADSKALDLLDKMLTFNPHKRIEVEQALAHPYLEQYYDPSDEPIAEAPFKFDMELDDLPKEKLKELIFEETARFQPGYRS
La longitud de la secuancia es: 360 residuos.


### BLASTp de la secuencia en el PDB
Descomentar el código para volver a ejecutar, o en su defecto, recuperar el archivo de blast guardado en el directorio.

In [9]:
## Hacemos un blast
# blast_record = blastPDB(seq_prot)
## Lo guardamos para poder usarlo y no tener que rehacerlo
# pickle.dump(blast_record, open(F'data/{prot_name}_blast_record.pkl', 'wb'))
# Para abrirlo:
blast_record = pickle.load(open(F'data/{prot_name}_blast_record.pkl', 'rb'))

Se obtienen los "hits" del BLAST según un valor de identidad especificado. También se obtiene una lista con los identificadores de dichos hits.

In [10]:
identidad  = 95
pdbids = blast_record.getHits(percent_identity = identidad) # Devuelve un diccionario con cada proteína hit
len(pdbids) # 391 proteínas tienen un 95% de identidad con la seucuencia
pdbids_list = list(pdbids.keys()) # lista de pdb_ids de las estructuras de CDK2

print(F'Se obtuvo un total de {len(pdbids_list)} hits con una identidad del {identidad}%')

Se obtuvo un total de 166 hits con una identidad del 95%


### Lista de identificadores de los PDB hits
Se guarda un archivo csv con todos los identificadores. Descomentar el código para volver a guardar.  

In [10]:
# Guarda la lista de PDB IDs de los hits
import csv
num_pdb_ids = len(pdbids_list); num_pdb_ids
with open(F'data/{prot_name}_pdb_{str(num_pdb_ids)}_IDs_{uniprot_id}.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting = csv.QUOTE_ALL)
    wr.writerow(pdbids_list)

## Selección de los residuos con estructura secundaria
Basado en la selección de residuos usados por Pisani (2016) al trabajar con la proteína CDK2.  
La idea es utilizar la estructura secundaria de la proteína para hecer la superposición de las estructuras.

In [152]:
ref_prot_id = pdbids_list[1] # Seleccionamos un pdb id de la proteina
print(F'Estructura de referencia usada: {ref_prot_id}')
cabezal = parsePDB(ref_prot_id, header=True, model=0)
estructura = parsePDB(ref_prot_id, folder = 'data')
sec_sctr = assignSecstr(cabezal, estructura.select('protein')).getSecstrs()
sec_sctr_HE = estructura.select(F'secondary H E and calpha and resnum 1:{len(seq_prot)}').getResnums()

@> PDB file is found in working directory (4fmq.pdb.gz).
@> PDB file is found in working directory (data/4fmq.pdb).
@> 3188 atoms and 1 coordinate set(s) were parsed in 0.04s.
@> Secondary structures were assigned to 224 residues.


Estructura de referencia usada: 4fmq


**Finalmente, obtenemos la lista de residuos que pertenecen a alguna región con estructura secundaria en la proteína.**

In [153]:
sec_sctr_res_list = sec_sctr_HE.tolist()
sec_sctr_res_str = ' '.join(map(str, sec_sctr_res_list))
sec_sctr_res_str

'13 14 17 18 25 26 27 28 29 30 31 32 33 38 39 40 41 42 43 44 49 50 51 52 53 54 55 56 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 88 89 90 101 102 103 104 105 106 110 111 112 113 114 115 116 117 118 119 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 145 146 155 156 157 163 164 165 172 173 195 196 197 198 199 200 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 232 233 234 235 236 237 238 239 240 241 242 243 244 245 257 258 259 260 261 262 263 264 265 266 267 274 275 276 277 278 279 283 284 285 286 287 288 289 290 291 292 293 294 303 304 305 306 307 308 309 339 340 341 342 343 344 345 346 347 348 349 350 351 352'

## Descarga de los archivos PDB
El siguiente ciclo descarga cada uno de los archivos PDB de la lista de identificadores PDB.
* Descarga el archivo PDB completo
* Descarga únicamente la cadena protéica del PDB con mayor identidad a la secuencia de UniProt dada (**Quedan alineadas por su estructura secundaria**)
* Descarga cualquier ligando asociado a la cadena protéica de mayor identidad.

In [ ]:
# Directorios
DIR_OUT_RAW_PDBS = '../ARCHIVOS/CRISTALES/PDB_ERK2_RAW_files/'
DIR_OUT_ERK2_CHAINS = '../ARCHIVOS/CRISTALES/PROT_ERK2_CHAINS/'
DIR_OUT_ERK2_LIGS = '../ARCHIVOS/CRISTALES/LIGS_ERK2/RAW/'

# Crea los directorios si no existen
import os
for dir in [DIR_OUT_RAW_PDBS, DIR_OUT_ERK2_CHAINS, DIR_OUT_ERK2_LIGS]
    if not os.path.exist(dir):
        os.makedirs(dir)

In [ ]:
# Estructura de referencia para alinear los PDBs, se usarán los CA de los residuos que pertenezcan a una región con estructura secundaria.
best_id = blast_record.getBest()['pdb_id']
chain_best_id = pdbids[best_id]['chain_id']
ref_struct = parsePDB(best_id, folder = DIR_OUT_RAW_PDBS).select('protein and chain ' + 
                                                                 chain_best_id)

for pdb_id in pdbids_list:
    try:
        chain_id = pdbids[pdb_id]['chain_id'] # Obtiene el id de la cadena correspondiente a la mejor estructura empatada según el BLAST
        pdb_cry = parsePDB(pdb_id, folder = DIR_OUT_RAW_PDBS) # Obtiene del pdb la estructura y la guarda en el folder
        pdb_chain = pdb_cry.select('protein and chain ' + chain_id) # selecciona de la proteína a la cadena que el BLAST indicó
          # Realiza el alineamiento usando los CA de los residuos con estructura Secundaria
        pdb_alg = matchAlign(pdb_chain, ref_struct, 
                             overlap=85, tarsel='calpha and resnum' + ' ' + sec_sctr_res_str) 
        # Retorna un tuple con la estructura alineada en el primer índice
        protein = pdb_alg[0]
    except Exception as e:
        print(e, "Error al alinear:", pdb_id)
        continue
    else:
        repr(protein)
        if protein: # Si hubo una cadena, la guarda
            protein.setChids("A") # fuerza que la cadena sea renombrada a "A", para homogenizar
            writePDB(DIR_OUT_ERK2_CHAINS + pdb_id + '_A.pdb', protein) # Guarda el archivo
        # Comprueba si existen ligandos (moléculas no protéicas)
        ligand = pdb_cry.select('not protein and not water and chain ' + chain_id)
        repr(ligand)
        if ligand: # Si hubo un ligando, lo guarda
            writePDB(DIR_OUT_ERK2_LIGS + pdb_id + '_lig.pdb', ligand)

# ¡Terminado!